In [2]:
import os
import pandas as pd
import xlrd  # dong nay de doc file xlsm
import glob
pd.set_option('display.max_columns',500,'float_format', '{:,.2f}'.format) #hien het cac dong, format dau phay, 2 so 0
pd.set_option('display.max_rows', 500)
path_yearly=     '../data/raw/Bank/yearly/'   # use your path
#path_quarterly = '../data/raw/Bank/Q3/'  # use your path
path_quarterly = '../data/raw/Bank/Quy4/'
data_clean_bank_yearly = '../data/clean/bank/yearly/'
data_clean_bank_quarterly = '../data/clean/bank/quarterly/'

# thay đổi path ở dòng path và data_clean để lấy dữ liệu yearly hoặc quarterly
path = path_quarterly
data_clean=   data_clean_bank_quarterly
report_files = glob.glob(path + "*.xlsm")

list_BS = []
list_IS = []
list_CF = []
list_FS = []
list_notes = []
list_growth = []
list_tindung_growth_CP = []

for filename in report_files:
    df = pd.read_excel(filename, sheet_name=None, header=None)
    df_BS = df['Cân đối kế toán'] # lay sheet CDKT
    Ma_CP = df_BS.loc[7,1]   # lay ma cp o dong 8, cot B
    df_BS = df['Cân đối kế toán'] # lay sheet CDKT
    Ma_CP = df_BS.loc[7,1]   # lay ma cp o dong 8, cot B
    Ma_CP
    #df_BS.dropna(how='all', inplace = True)
    row_tieu_de = int(df_BS[df_BS[0] == 'Tiêu đề'].index[0])# tim row Tiêu đề
    begin_BS = row_tieu_de + 6
    row_CTCP_Fin_row = int(df_BS[df_BS[0] == 'CÔNG TY CỔ PHẦN FIINGROUP'].index[0])# tim row CÔNG TY CỔ PHẦN FIINGROUP
    end_BS = row_CTCP_Fin_row - 1
    BS = df_BS.iloc[begin_BS:end_BS]
    BS.iloc[0,0]= "items"
    BS.iloc[1,0]= "Balance_sheet"
    BS.columns = BS.iloc[0] # lay dong dau tien lam ten column
    BS.drop(BS.index[0],inplace= True) # bo dong dau tien
    BS.set_index('items', inplace = True) # set index cột items
    #BS_drop = BS.drop('2022F',axis=1) # loại cột Forecást 2022
    #BS.loc['Tín dụng'] = BS.loc['Cho vay khách hàng'] + BS.loc['Tiền gửi tại các TCTD khác và cho vay các TCTD khác']
    BS.loc['Tín dụng growth'] = BS.loc['Cho vay khách hàng'].pct_change(axis = 'rows',periods=3)*100  # tin dung 2 quy

    idx = 0
    # the

    BS_T = BS.T
    BS_T.insert(loc = idx, column= 'CP', value= Ma_CP) # them cot CP o cot so 1
    list_BS.append(BS_T)
    BS_pct= BS.loc['TỔNG TÀI SẢN':].pct_change(axis = 'columns')*100
    #BS_pct
    BS_growth = BS_pct.loc[['TỔNG TÀI SẢN','     Cho vay khách hàng','     Tiền gửi của khách hàng','Vốn chủ sở hữu','Tín dụng growth']]

    #%% INCOME STATEMENT
    df_IS = df['Kết quả Kinh doanh'] # Kết quả Kinh doanh
    row_tieu_de = int(df_IS[df_IS[0] == 'Tiêu đề'].index[0])# tim row Tiêu đề
    begin_IS = row_tieu_de + 6
    row_CTCP_Fin_row = int(df_IS[df_IS[0] == 'CÔNG TY CỔ PHẦN FIINGROUP'].index[0])# tim row CÔNG TY CỔ PHẦN FIINGROUP
    end_IS = row_CTCP_Fin_row - 1
    IS = df_IS.iloc[begin_IS:end_IS]
    IS.iloc[0,0]= "items"
    IS.iloc[1,0] = "Income_statement"
    IS.columns = IS.iloc[0] # lay dong dau tien lam ten column
    IS.drop(IS.index[0],inplace= True) # bo dong dau tien
    #FS_T.insert(loc = idx, column= 'CP', value= Ma_CP) # them cot CP o cot so 1
    IS.set_index('items', inplace = True) #
    #IS_drop = IS.drop('2022F',axis=1) # loại cột Forecást 2022
    IS_T = IS.T
    IS_T.insert(loc = idx, column= 'CP', value= Ma_CP) # them cot CP o cot so 1
    list_IS.append(IS_T)
    IS_pct= IS.loc['Thu nhập lãi và các khoản thu nhập tương tự':,'Q2/2020':].pct_change(axis = 'columns')*100
    #IS_pct
    IS_growth = IS_pct.loc[['Thu nhập lãi thuần','     Thu nhập từ hoạt động dịch vụ','Chi phí hoạt động','Tổng thu nhập hoạt động','LN thuần từ hoạt động kinh doanh trước CF dự phòng rủi ro tín dụng','Tổng lợi nhuận trước thuế','Lợi nhuận sau thuế','Lãi cơ bản trên cổ phiếu (VND)']]
    #IS_growth
    ratio_growth = pd.concat([BS_growth, IS_growth], axis=0)
    ratio_growth.rename_axis("ratio growth", inplace=True)
    growth =  ratio_growth.T
    #growth
    idx = 0
    growth.insert(loc = idx, column= 'CP', value= Ma_CP)
    #growth
    list_growth.append(growth)
    #CASHFLOW DIRECT
    df_CF = df['Lưu chuyển tiền tệ'] # Lưu chuyển tiền tệ
    row_tieu_de = int(df_CF[df_CF[0] == 'Tiêu đề'].index[0]) #tim tieu de
    begin_CF = row_tieu_de + 6
    #row_gian_tiep = int(df_CF[df_CF[0] == 'Gián Tiếp '].index[0])# tim
    #row_CTCP_Fin_row = int(df_CF[df_CF[0] == 'CÔNG TY CỔ PHẦN FIINGROUP'].index[0])# tim row CÔNG TY CỔ PHẦN FIINGROUP
    #end_CF = row_CTCP_Fin_row - 1
    row_end_CF_dir = int(df_CF[df_CF[0] == 'Tiền và các khoản tương đương tiền tại thời điểm cuối kỳ'].index[0]) # tim row cuối CF Direct
    CF_Dir = df_CF.iloc[begin_CF:row_end_CF_dir + 1] # Cashflow statement truc tiep
    CF_Dir.iloc[0,0]= "items"
    CF_Dir.iloc[1,0]= "CF trực tiếp"
    CF_Dir.columns = CF_Dir.iloc[0] # lay dong dau tien lam ten column
    CF_Dir.drop(CF_Dir.index[0],inplace= True) # bo dong dau tien
    CF_Dir.set_index('items', inplace = True) #
    CF_T = CF_Dir.T
    CF_T.insert(loc = idx, column= 'CP', value= Ma_CP) # them cot CP o cot so 1
    list_CF.append(CF_T)
    # CASHFLOW INDIRECT
    #CF_inDir = df_CF.iloc[row_gian_tiep:end_CF] # Cashflow statement gian tiep
    #CF_inDir.iloc[0,0]= "items"
    #CF_inDir.iloc[1,0]= "CF gián tiếp"
    #CF_inDir.columns = CF_inDir.iloc[0] # lay dong dau tien lam ten column
    #CF_inDir.drop(CF_inDir.index[0],inplace= True) # bo dong dau tien
    #CF_inDir.set_index('items', inplace = True) #
    #CF_inDir
    # notes
    df_note = df['Thuyết minh'] # Thuyết minh
    row_tieu_de = int(df_note[df_note[0] == 'Tiêu đề'].index[0]) #tim tieu de
    begin_note = row_tieu_de + 6
    row_CTCP_Fin_row = int(df_note[df_note[0] == 'CÔNG TY CỔ PHẦN FIINGROUP'].index[0])# tim row CÔNG TY CỔ PHẦN FIINGROUP
    end_note = row_CTCP_Fin_row - 1
    notes = df_note.iloc[begin_note:end_note] # C
    notes.iloc[1,0]= "Notes"
    notes.iloc[0,0]= "items"
    notes.iloc[1,0]= "notes"
    notes.columns = notes.iloc[0] # lay dong dau tien lam ten column
    notes.drop(notes.index[0],inplace= True) # bo dong dau tien
    notes.set_index('items', inplace = True) #
    notes_T = notes.T
    notes_T.insert(loc = idx, column= 'CP', value= Ma_CP) # them
    list_notes.append(notes_T)
    FS = pd.concat([BS, IS, CF_Dir,notes], axis=0)
    #FS.loc['CP'] = Ma_CP # them dong co phieu
    #FS.index = FS.index + 1
    #FS.sort_index(inplace = True)
    FS.to_excel(data_clean + Ma_CP + '_clean.xlsx')
    FS.index = FS.index.str.strip() # cat cac space o dau va cuoi str
    FS
    FS_T = FS.T # BS.transpose() trandoi col to row
    #BS_T.columns = BS_T.iloc[0] # lay dong dau tien lam ten column
    #BS_T.drop(BS_T.index[0],inplace= True) # bo dong dau tien
    #idx = 0
    FS_T.insert(loc = idx, column= 'CP', value= Ma_CP) # them cot CP o cot so 1
    FS_T.to_excel(data_clean + Ma_CP + '_clean_transpose.xlsx')
    list_FS.append(FS_T)
    #FS_all = pd.concat([FS_all,FS_T], axis = 0, ignore_index = True)
FS_all = pd.concat(list_FS, axis = 0 )
BS_all = pd.concat(list_BS, axis = 0 )
IS_all = pd.concat(list_IS, axis = 0 )
CF_all = pd.concat(list_CF, axis = 0 )
notes_all = pd.concat(list_notes, axis = 0 )
FS_all.rename_axis("BCTC_nganh", inplace = True)
BS_all.rename_axis("BS_nganh", inplace = True)
IS_all.rename_axis("IS_nganh", inplace = True)
CF_all.rename_axis("CF_nganh", inplace = True)
FS_all.to_excel(data_clean + 'FS_all' + '_clean_transpose.xlsx')

growth_rate = pd.concat(list_growth, axis = 0 )
growth_rate

#FS_all


/var/folders/1v/qp6912f958xf055vcq9kbnm00000gn/T/ipykernel_72110/3849462718.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BS.iloc[0,0]= "items"
/var/folders/1v/qp6912f958xf055vcq9kbnm00000gn/T/ipykernel_72110/3849462718.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BS.iloc[1,0]= "Balance_sheet"
/var/folders/1v/qp6912f958xf055vcq9kbnm00000gn/T/ipykernel_72110/3849462718.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

KeyError: 'Q2/2020'

In [ ]:
#group_time =  FS_all.groupby(FS_all.index)['Lợi nhuận sau thuế'].sum()
#quaterly_all = pd.DataFrame(group_time)
#quaterly_all
#BS_T.drop(BS_T.index[0],inplace= True) # bo dong dau tien
#FS_all.drop('TTM',inplace= True) # bo dong dau tien
#FS_all
#FS_all[['Q','year']] = FS_all.index[0].split('/')
#FS_all['Q'] = FS_all.index.str.split('/',0, expand = True).str.len()
FS_all.dropna(how='all', inplace = True)
FS_all['Q'] = FS_all.index.str.split('/').str[0]
FS_all['year'] = FS_all.index.str.split('/').str[1]
FS_all['time'] = FS_all['year']+FS_all['Q']
BS_all['Q'] = BS_all.index.str.split('/').str[0]
BS_all['year'] = BS_all.index.str.split('/').str[1]
BS_all['time'] = BS_all['year']+BS_all['Q']
IS_all['Q'] = IS_all.index.str.split('/').str[0]
IS_all['year'] = IS_all.index.str.split('/').str[1]
IS_all['time'] = IS_all['year']+IS_all['Q']
notes_all['Q'] = notes_all.index.str.split('/').str[0]
notes_all['year'] = notes_all.index.str.split('/').str[1]
notes_all['time'] = notes_all['year']+notes_all['Q']
#a
#group_time =  FS_all.groupby('time')['Các khoản cho vay phân loại theo chất lượng nợ cho vay','Nợ dưới tiêu chuẩn','Nợ nghi ngờ','Nợ xấu có khả năng mất vốn','Thu nhập lãi và các khoản thu nhập tương tự','Chi phí lãi và các chi phí tương tự','Thu nhập lãi thuần','Tổng thu nhập hoạt động','Chi phí hoạt động','Chi phí dự phòng rủi ro tín dụng','Lợi nhuận sau thuế'].sum()
#quaterly_all = pd.DataFrame(group_time)
#R= quaterly_all.iloc[:,[0,1,2,3,4,7,8,9,10,11,12,13]].loc['2020Q1':].T
#R
#.loc['2020Q1':'2022Q2']
#0,1,3,5,6,7,8,9
#FS_all[['Q','year']]
#FS_all['Q']
#b = pd.to_datetime(FS_all.index[0])
#a = FS_all.index[0]
#a = '2022Q1'
#b = pd.to_datetime(a)
#b.quarter



In [ ]:
# tinh tong nganh groupby
notes_measurement = notes_all.groupby('time').sum().T
notes_nganh_total = notes_all.groupby('time').sum().T
#IS_measurement = IS_all.groupby('time')['Thu nhập lãi và các khoản thu nhập tương tự','Chi phí lãi và các chi phí tương tự','Thu nhập lãi thuần','Tổng thu nhập hoạt động','Chi phí hoạt động','Chi phí dự phòng rủi ro tín dụng','Lợi nhuận sau thuế'].sum().T

IS_nganh_total = IS_all.groupby('time').sum().T
BS_nganh_total = BS_all.groupby('time').sum().T
IS_TTM_nganh = IS_nganh_total.loc['Thu nhập lãi và các khoản thu nhập tương tự':'Lãi cơ bản trên cổ phiếu (VND)'].astype(float).rolling(4, axis = 1).sum()
notes_nganh_total
IS_CS_nganh = IS_TTM_nganh*100/IS_TTM_nganh.loc['Tổng thu nhập hoạt động']
IS_CS_nganh
IS_nganh_total_T = IS_nganh_total.T
#IS_nganh_total_T[['Thu nhập lãi và các khoản thu nhập tương tự','Tổng thu nhập hoạt động','Lợi nhuận sau thuế']].T
#BS_nganh_total.loc['Tín dụng tổng'] = BS_nganh_total.loc['Cho vay khách hàng'] + BS_nganh_total.loc['Tiền gửi tại các TCTD khác và cho vay các TCTD khác']
tindung_growh_nganh_2Q = BS_nganh_total.loc['Cho vay khách hàng'].pct_change(axis = 'rows',periods=3)*100

tindung_growh_nganh_2Q['2022Q3']

In [ ]:
#IS_all_nganh = IS_all.groupby('time').sum().T
#TTM = IS_all_nganh.rolling(4, axis = 1).sum()
#TTM
BS_TS_nganh_BQ = BS_nganh_total.loc['TỔNG TÀI SẢN'].rolling(4, axis = 0).mean() # Tổng TS bình quân để tính chỉ số hiệu quả
BS_TSSL= BS_nganh_total.loc[['Tiền gửi tại Ngân hàng nhà nước Việt Nam','     Tiền gửi tại các TCTD khác','     Cho vay khách hàng','     Chứng khoán đầu tư sẵn sàng để bán','     Chứng khoán đầu tư giữ đến ngày đáo hạn']].sum() # TS sinh lãi bình quân
BS_TSSL_BQ = BS_TSSL.rolling(4, axis = 0).mean() # Tài sản sinh lãi bình quân (theo năm)
BS_VCSH_BQ = BS_nganh_total.loc['Vốn chủ sở hữu'].rolling(4, axis = 0).mean() # Tổng TS bình quân để tính chỉ số hiệu quả
HQ_TSBQ = IS_TTM_nganh.loc[['Tổng thu nhập hoạt động','Chi phí hoạt động']]*100/BS_TS_nganh_BQ    # chỉ số Hiệu Quả trên TSBQ
HQ_TSBQ.rename_axis("perfomance_nganh", inplace = True)
HQ_TSBQ.rename(index= {'Tổng thu nhập hoạt động':'Tổng TN HĐ/Tổng TS BQ (%)', 'Chi phí hoạt động':'Chi phí hoạt động/Tổng TS BQ(%)'}, inplace = True)
HQ_TSBQ.loc['TS SinhLai BQ/TSBQ'] =  BS_TSSL_BQ*100/BS_TS_nganh_BQ  # TS SinhLai BQ/TSBQ
HQ_TSBQ.loc['Yield on Earning Asset - YOEA'] = IS_TTM_nganh.loc['Thu nhập lãi và các khoản thu nhập tương tự']*100/BS_TSSL_BQ # Yield on Earning Asset - tỷ suất sinh lời trên các tài sản có sinh lãi
HQ_TSBQ.loc['Chi phí Lãi'] = IS_TTM_nganh.loc['Chi phí lãi và các chi phí tương tự']*100/BS_TSSL_BQ
HQ_TSBQ.loc['NIM'] = IS_TTM_nganh.loc['Thu nhập lãi thuần']*100/BS_TSSL_BQ
HQ_TSBQ.loc['chi phí trích lập DP/TSSLBQ'] = IS_TTM_nganh.loc['Chi phí dự phòng rủi ro tín dụng']*100/BS_TSSL_BQ
HQ_TSBQ.loc['LN thuần/TS SinhLai BQ'] =  IS_TTM_nganh.loc['Lợi nhuận sau thuế']*100/BS_TSSL_BQ
HQ_TSBQ.loc['ROAA'] =  IS_TTM_nganh.loc['Lợi nhuận sau thuế']*100/BS_TS_nganh_BQ
HQ_TSBQ.loc['ROAE'] =  IS_TTM_nganh.loc['Lợi nhuận sau thuế']*100/BS_VCSH_BQ
HQ_TSBQ.loc['Leverage_BQ'] =  BS_TS_nganh_BQ/BS_VCSH_BQ
HQ_TSBQ.loc['Thu nhập lãi thuần/Tổng TN Hoạt động (%)'] = IS_CS_nganh.loc['Thu nhập lãi thuần']
HQ_TSBQ.loc['Lãi/Lỗ thuần từ hoạt động dịch vụ/Tổng TN Hoạt động (%)'] = IS_CS_nganh.loc['Lãi/Lỗ thuần từ hoạt động dịch vụ']
HQ_TSBQ.loc['Lãi/(lỗ) thuần từ hoạt động khác/Tổng TN Hoạt động (%)'] = IS_CS_nganh.loc['Lãi/(lỗ) thuần từ hoạt động khác']
HQ_TSBQ.loc['CIR - Chi phí hd/Tổng TN hd (%)'] = IS_CS_nganh.loc['Chi phí hoạt động']
HQ_TSBQ.loc['LN thuần/Tổng TN Hoạt động (%)'] = IS_CS_nganh.loc['Lợi nhuận sau thuế']
#HQ_TSBQ.loc['CAR (%)'] = notes_nganh_total.loc['CAR'].astype(float)*100  # chuyen string sang float, do trong note, dong CAR ở dạng string
HQ_TSBQ.loc['VCSH/TS end (%)'] = BS_nganh_total.loc['Vốn chủ sở hữu']*100/BS_nganh_total.loc['TỔNG TÀI SẢN']
HQ_TSBQ.loc['NPL (%)'] = notes_nganh_total.loc[['     Nợ xấu có khả năng mất vốn','     Nợ nghi ngờ','     Nợ dưới tiêu chuẩn']].sum()*100/notes_nganh_total.loc['Các khoản cho vay phân loại theo chất lượng nợ cho vay']
HQ_TSBQ.loc['LLR (%)'] = BS_nganh_total.loc['     Dự phòng rủi ro cho vay khách hàng']*(-100)/notes_nganh_total.loc[['     Nợ xấu có khả năng mất vốn','     Nợ nghi ngờ','     Nợ dưới tiêu chuẩn']].sum()
HQ_TSBQ.loc['LDR (%)'] = BS_nganh_total.loc['Cho vay khách hàng']*100/BS_nganh_total.loc[['     Tiền gửi của khách hàng','     Phát hành giấy tờ có giá']].sum()
HQ_TSBQ.loc['CASA (%)'] = notes_nganh_total.loc[['     Tiền gửi không kỳ hạn','     Tiền gửi ký quỹ']].sum()*100/notes_nganh_total.loc['Các khoản tiền gửi phân loại theo loại tiền gửi']
HQ_TSBQ.loc['BVPS'] = BS_nganh_total.loc['Vốn chủ sở hữu']*10/BS_nganh_total.loc['          Vốn điều lệ']
HQ_TSBQ

In [ ]:
latest = '2022Q3'
BS_4Q_latest = BS_all[BS_all['time'] > '2021Q3']  #lay 4 quy gan nhat  , khong the dung loc vi index trung nhau, nen phai filter
IS_4Q_latest = IS_all[IS_all['time'] > '2021Q3']

BS_TTM_CP = BS_4Q_latest.groupby('CP').mean().T
IS_TTM_CP = IS_4Q_latest.groupby('CP').sum().T

BS_1Q_latest = BS_all[BS_all['time'] == '2022Q3'].set_index('CP', drop=True)
notes_1Q_latest = notes_all[notes_all['time'] == '2022Q3'].set_index('CP', drop=True)
BS_1Q_latest.sort_index(ascending=True,inplace = True)
BS_1Q_latest_T = BS_1Q_latest.T

notes_1Q_latest.sort_index(ascending=True,inplace = True)
notes_1Q_latest_T = notes_1Q_latest.T

BS_4Q_latest_T = BS_4Q_latest.T
BS_4Q_latest_T.loc['Tín dụng tổng'] = BS_4Q_latest_T.loc['Cho vay khách hàng'] + BS_4Q_latest_T.loc['Tiền gửi tại các TCTD khác và cho vay các TCTD khác']
#tindung_growh_cp_2Q =


#notes_1Q_latest.loc[['     Nợ xấu có khả năng mất vốn','     Nợ nghi ngờ','     Nợ dưới tiêu chuẩn']].sum()*100
#BS_1Q_latest.duplicated()
#df_daily['Date/Time'] = pd.to_datetime(df_daily['Date/Time'])
#growth_rate[growth_rate['CP'] == 'ACB ']

In [ ]:
#IS_TTM_CP
#IS_all
#measurement = pd.concat([notes_time, IS_time], axis=0)
#measurement.head()
#measurement.loc[:,'2020Q1':]
BS_TS_CP_BQ = BS_TTM_CP.loc['TỔNG TÀI SẢN']# Tổng TS bình quân để tính chỉ số hiệu quả
BS_TSSL_CP_BQ= BS_TTM_CP.loc[['Tiền gửi tại Ngân hàng nhà nước Việt Nam','     Tiền gửi tại các TCTD khác','     Cho vay khách hàng','     Chứng khoán đầu tư sẵn sàng để bán','     Chứng khoán đầu tư giữ đến ngày đáo hạn']].sum() # TS sinh lãi bình quân
BS_VCSH_CP_BQ = BS_TTM_CP.loc['Vốn chủ sở hữu'] #TS bình quân để tính chỉ số hiệu quả
HQ_TS_CP_BQ = IS_TTM_CP.loc[['Tổng thu nhập hoạt động','Chi phí hoạt động']]*100/BS_TS_CP_BQ    # chỉ số Hiệu Quả trên TSBQ
HQ_TS_CP_BQ.rename_axis("perfomance_CP", inplace = True)
HQ_TS_CP_BQ.rename(index= {'Tổng thu nhập hoạt động':'Tổng TN HĐ/Tổng TS BQ (%)', 'Chi phí hoạt động':'Chi phí hoạt động/Tổng TS BQ(%)'}, inplace = True)
HQ_TS_CP_BQ.loc['TS SinhLai BQ/TSBQ'] =  BS_TSSL_CP_BQ*100/BS_TS_CP_BQ  # TS SinhLai BQ/TSBQ
HQ_TS_CP_BQ.loc['Yield on Earning Asset - YOEA'] = IS_TTM_CP.loc['Thu nhập lãi và các khoản thu nhập tương tự']*100/BS_TSSL_CP_BQ # Yield on Earning Asset - tỷ suất sinh lời trên các tài sản có sinh lãi
HQ_TS_CP_BQ.loc['Chi phí Lãi'] = IS_TTM_CP.loc['Chi phí lãi và các chi phí tương tự']*100/BS_TSSL_CP_BQ
HQ_TS_CP_BQ.loc['NIM'] = IS_TTM_CP.loc['Thu nhập lãi thuần']*100/BS_TSSL_CP_BQ
HQ_TS_CP_BQ.loc['chi phí trích lập DP/TSSLBQ'] = IS_TTM_CP.loc['Chi phí dự phòng rủi ro tín dụng']*100/BS_TSSL_CP_BQ
HQ_TS_CP_BQ.loc['LN thuần/TS SinhLai BQ'] =  IS_TTM_CP.loc['Lợi nhuận sau thuế']*100/BS_TSSL_CP_BQ
HQ_TS_CP_BQ.loc['ROAA'] =  IS_TTM_CP.loc['Lợi nhuận sau thuế']*100/BS_TSSL_CP_BQ
HQ_TS_CP_BQ.loc['ROAE'] =  IS_TTM_CP.loc['Lợi nhuận sau thuế']*100/BS_VCSH_CP_BQ
HQ_TS_CP_BQ.loc['Leverage_BQ'] =  BS_TS_CP_BQ/BS_VCSH_CP_BQ
HQ_TS_CP_BQ.loc['Thu nhập lãi thuần/Tổng TN Hoạt động (%)'] = IS_TTM_CP.loc['Thu nhập lãi thuần']*100/IS_TTM_CP.loc['Tổng thu nhập hoạt động']
HQ_TS_CP_BQ.loc['Lãi/Lỗ thuần từ hoạt động dịch vụ/Tổng TN Hoạt động (%)'] = IS_TTM_CP.loc['Lãi/Lỗ thuần từ hoạt động dịch vụ']*100/IS_TTM_CP.loc['Tổng thu nhập hoạt động']
HQ_TS_CP_BQ.loc['Lãi/(lỗ) thuần từ hoạt động khác/Tổng TN Hoạt động (%)'] = IS_TTM_CP.loc['Lãi/(lỗ) thuần từ hoạt động khác']*100/IS_TTM_CP.loc['Tổng thu nhập hoạt động']
HQ_TS_CP_BQ.loc['CIR - Chi phí hd/Tổng TN hd (%)'] = IS_TTM_CP.loc['Chi phí hoạt động']*100/IS_TTM_CP.loc['Tổng thu nhập hoạt động']
HQ_TS_CP_BQ.loc['LN thuần/Tổng TN Hoạt động (%)'] = IS_TTM_CP.loc['Lợi nhuận sau thuế']*100/IS_TTM_CP.loc['Tổng thu nhập hoạt động']
#HQ_TS_CP_BQQ.loc['CAR (%)'] = notes_nganh_total.loc['CAR'].astype(float)*100  # chuyen string sang float, do trong note, dong CAR ở dạng string
HQ_TS_CP_BQ.loc['VCSH/TS end (%)'] = BS_1Q_latest_T.loc['Vốn chủ sở hữu']*100/BS_1Q_latest_T.loc['TỔNG TÀI SẢN']
HQ_TS_CP_BQ.loc['NPL (%)'] = notes_1Q_latest_T.loc[['     Nợ xấu có khả năng mất vốn','     Nợ nghi ngờ','     Nợ dưới tiêu chuẩn']].sum()*100/notes_1Q_latest_T.loc['Các khoản cho vay phân loại theo chất lượng nợ cho vay']
HQ_TS_CP_BQ.loc['LLR (%)'] = BS_1Q_latest_T.loc['     Dự phòng rủi ro cho vay khách hàng']*(-100)/notes_1Q_latest_T.loc[['     Nợ xấu có khả năng mất vốn','     Nợ nghi ngờ','     Nợ dưới tiêu chuẩn']].sum()
HQ_TS_CP_BQ.loc['LDR (%)'] = BS_1Q_latest_T.loc['Cho vay khách hàng']*100/BS_1Q_latest_T.loc[['     Tiền gửi của khách hàng','     Phát hành giấy tờ có giá']].sum()
HQ_TS_CP_BQ.loc['CASA (%)'] = notes_1Q_latest_T.loc[['     Tiền gửi không kỳ hạn','     Tiền gửi ký quỹ']].sum()*100/notes_1Q_latest_T.loc['Các khoản tiền gửi phân loại theo loại tiền gửi']
HQ_TS_CP_BQ.loc['BVPS'] = BS_1Q_latest_T.loc['Vốn chủ sở hữu']*10/BS_1Q_latest_T.loc['          Vốn điều lệ']
HQ_TS_CP_BQ.loc['credit growth'] =  BS_1Q_latest_T.loc['Tín dụng growth']
HQ_TS_CP_BQ['NGANH'] = HQ_TSBQ['2022Q3']
HQ_TS_CP_BQ.loc['credit growth','NGANH'] = tindung_growh_nganh_2Q['2022Q3']


In [ ]:
HQ_TS_CP_BQ.sort_values(by = 'ROAE', axis = 1, ascending = False, inplace=True)
HQ_TS_CP_BQ.to_excel(data_clean +'NH_ratios_TTM.xlsx')
HQ_TS_CP_BQ


In [ ]:
IS_nganh_total